# Correccion de datos

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()


In [2]:
df_original = pd.read_csv('../data/train.csv')
drop_columns = ['centroscomercialescercanos', 'escuelascercanas', 'usosmultiples', 'gimnasio',
               'fecha', 'lng', 'lat', 'idzona', 'id', 'direccion', 'titulo', 'descripcion', 'ciudad']
df = df_original.drop(drop_columns, axis=1)

In [3]:
def generar_resumen(dforiginal, variable):
    df = dforiginal[['tipodepropiedad', variable]].copy()
    resumen = pd.DataFrame({
            "con": df[~df[variable].isnull()].groupby('tipodepropiedad').size(),
            "sin": df[df[variable].isnull()].groupby('tipodepropiedad').size(),
            "porcentaje_no": df[df[variable].isnull()].groupby('tipodepropiedad').size() * (100/df.groupby('tipodepropiedad').size()),
            "promedio_del_valor": df.groupby('tipodepropiedad')[variable].mean(),
            "moda": df.groupby('tipodepropiedad')[variable].agg(lambda x:x.mode()[0] if len(x.mode()) == 1 else 0),
        }, index=df[df[variable].isnull()].groupby('tipodepropiedad').size().index)

    return resumen

In [4]:
def reemplazar_por_metrica(df, variable, resumen, metrica):
    for p in resumen.index:
        df.loc[(df.tipodepropiedad == p) & (df[variable].isnull()), variable] = resumen.loc[p, metrica]
        
def reemplazar_por_moda(df, variable, resumen):
    reemplazar_por_metrica(df, variable, resumen, 'moda')
    
def reemplazar_por_promedio(df, variable, resumen):
    reemplazar_por_metrica(df, variable, resumen, 'promedio_del_valor')

## Propiedades sin tipodepropiedad (506 propiedades con NA y 156 sin provincia)

Se excluyen porque son un subconjunto muy pequeño respecto del dataset total.

In [5]:
df.dropna(subset=['tipodepropiedad'], inplace=True)

## Metricas utilizadas

El resumen que se sacara de cada variable cuenta con el total de propiedades que tienen dicha variable con un valor no nulo, la cantidad que tiene el valor nulo y el porcentaje de estas sobre el total de las propiedades de ese tipo.

Otra metrica util es el promedio del valor de esa variable entre las propiedades que tienen un valor no nulo.

La ultima metrica sera la moda de esa variable. La moda es el valor que mas veces se repite dentro de esa variable, esta es una forma de asegurarnos de poner un valor que no rompa con la distribución de dicha variable. En el caso de tener 3 propiedades donde los valores sean [1, 1, 1, 1, 21] el promedio nos dará 5, pero el valor mas común es 1 lo cual 1  es mejor candidato a llenar aquellos datos que falten.
https://en.wikipedia.org/wiki/Mode_(statistics)

## Generación de todos los resumenes



In [6]:
import os

def load_or_create(df, variable):
    csv = "r_{}.csv".format(variable)
    if not os.path.exists(csv):
        resumen = generar_resumen(df, variable)
        resumen.to_csv(csv)
        return resumen
    
    return pd.read_csv(csv, index_col='tipodepropiedad')


resumen_garages = load_or_create(df, 'garages')
resumen_habitacion = load_or_create(df, 'habitaciones')
resumen_antiguedad = load_or_create(df, 'antiguedad')
resumen_banos = load_or_create(df, 'banos')



## Corrección de datos NA en el dataset

Hay muchos datos que faltan y son NA dentro del dataset. Aca se van a limpiar esos datos asumiendo algunas cosas. Cada limpieza va a tener una explicación de porque y justificación

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239954 entries, 0 to 239999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tipodepropiedad  239954 non-null  object 
 1   provincia        239801 non-null  object 
 2   antiguedad       196435 non-null  float64
 3   habitaciones     217501 non-null  float64
 4   garages          202220 non-null  float64
 5   banos            213763 non-null  float64
 6   metroscubiertos  222568 non-null  float64
 7   metrostotales    188496 non-null  float64
 8   piscina          239954 non-null  float64
 9   precio           239954 non-null  float64
dtypes: float64(8), object(2)
memory usage: 20.1+ MB


### Garage como NA

Aquellas propiedades que no tengan garage como valor numerico se les pondrá un 0. Se asume que muchas propiedades no tendrán garage y esta bien que no lo tengan, un terreno no tiene garage por ejemplo. Todo lo que sea NA se pasa a 0.



In [8]:
print("Total con:", resumen_garages.con.sum())
print("Total sin:", resumen_garages.sin.sum())
print("Porcentaje: {:.2f}%".format(resumen_garages.sin.sum()*(100/(resumen_garages.con.sum()+resumen_garages.sin.sum()))))
resumen_garages

Total con: 202218
Total sin: 37734
Porcentaje: 15.73%


,con,sin,porcentaje_no,promedio_del_valor,moda
tipodepropiedad,,,,,
Apartamento,53605,3736,6.515408,1.410242,1.0
Bodega comercial,881,525,37.339972,0.484677,0.0
Casa,117756,23961,16.907640,1.705926,2.0
Casa en condominio,16060,3237,16.774628,1.908157,2.0
Casa uso de suelo,468,240,33.898305,1.427350,2.0
Departamento Compartido,137,4,2.836879,1.401460,0.0
Duplex,336,7,2.040816,1.181548,1.0
Edificio,873,523,37.464183,0.725086,0.0
Huerta,15,5,25.000000,0.000000,0.0


In [9]:
total_garages_na = sum(df.garages.isnull())
print("Cantidad de propiedades sin dato del garage:", total_garages_na)

reemplazar_por_moda(df, 'garages', resumen_garages)

print("Quedaron {} con valor NA en garages".format(sum(df.garages.isnull())))

Cantidad de propiedades sin dato del garage: 37734
Quedaron 0 con valor NA en garages


### Baños como NA

No todas las propiedades tendrán 0 baños, pero aquellas que sean terrenos por ej tiene sentido que tengan 0 baños.

In [10]:
print("Total de propiedades sin baño", sum(df.banos.isnull()))

Total de propiedades sin baño 26191


Promedio de baños de las propiedades que tiene baño es: 2.1324
Promedio de baños segun propiedades es casi 2 en todos los casos, si los redondeo son todos 2 salvo dos.
Promedio de los promedios por propiedda 1,9857

Promedio de baños:                  2,1324
Promedio por propiedad:            ~2
Promedio por propiedad redondeado:  2 (salvo en 2 casos, 1 y 3)
Promedio pro propiedad promediado:  1,9857

In [11]:
# promedio
# df.loc[~df.banos.isnull(), ('tipodepropiedad', 'banos')].mean()

# promedio por propiedad
# df.loc[~df.banos.isnull(), ('tipodepropiedad', 'banos')].groupby('tipodepropiedad').mean()

# promedio por propiedad redondeado
# df.loc[~df.banos.isnull(), ('tipodepropiedad', 'banos')].groupby('tipodepropiedad').mean().round()

# promedio de todos los promedios
df.loc[~df.banos.isnull(), ('tipodepropiedad', 'banos')].groupby('tipodepropiedad').mean().mean()

banos    1.985744
dtype: float64

Misteriosamente todos los tipos de propiedades tiene como promedio 2 baños

In [12]:
resumen_banos

,con,sin,porcentaje_no,promedio_del_valor,moda
tipodepropiedad,,,,,
Apartamento,56004.0,1337,2.331665,1.816817,2.0
Bodega comercial,746.0,660,46.941679,1.847185,1.0
Casa,133108.0,8609,6.074783,2.240429,2.0
Casa en condominio,18494.0,803,4.161269,2.366065,2.0
Casa uso de suelo,551.0,157,22.175141,2.404719,2.0
Departamento Compartido,134.0,7,4.964539,1.791045,1.0
Duplex,338.0,5,1.457726,1.360947,1.0
Edificio,367.0,1029,73.710602,2.626703,4.0
Garage,NaN,1,100.000000,NaN,0.0


In [13]:
print("Cantidad de propiedades sin dato del baño:", sum(df.banos.isnull()))

reemplazar_por_moda(df, 'banos', resumen_banos)

print("Quedaron {} con valor NA en banos".format(sum(df.banos.isnull())))

Cantidad de propiedades sin dato del baño: 26191
Quedaron 0 con valor NA en banos


### Habitaciones como NA

Aqui terrenos y lotes no tendran habitaciones (no tienen estructura construida)

In [14]:
print("Total de propiedades sin habitaciones", sum(df.habitaciones.isnull()))

resumen_habitacion

Total de propiedades sin habitaciones 22453


,con,sin,porcentaje_no,promedio_del_valor,moda
tipodepropiedad,,,,,
Apartamento,56347.0,994,1.733489,2.386533,2.0
Bodega comercial,157.0,1249,88.833570,2.630573,1.0
Casa,138379.0,3338,2.355398,3.092080,3.0
Casa en condominio,19082.0,215,1.114163,2.957132,3.0
Casa uso de suelo,502.0,206,29.096045,4.284861,3.0
Departamento Compartido,137.0,4,2.836879,2.598540,3.0
Duplex,340.0,3,0.874636,2.541176,2.0
Edificio,303.0,1093,78.295129,6.128713,10.0
Garage,NaN,1,100.000000,NaN,0.0


#### Decisiones de como limpiar esto

Se puede ver que hay tipos de propiedades que la mayoria no tienen habitaciones y otras propiedades (departamentos, casas) la mayoria si tienen habitación. Por lo cual, aquellas propiedades donde la mayoria (arriba del 50%) no tienen, se asumira que fue un error al ingresar la informacion por lo que se le asignara 0 en habitaciones. En los otros casos se les colocara el promedio general de su tipo de propiedad para no romper la distribución de la información

In [15]:
print("Cantidad de propiedades sin dato del baño:", sum(df.habitaciones.isnull()))

sin_habitaciones_y_deberian = list(resumen_habitacion[resumen_habitacion.porcentaje_no < 50].index)

print(sin_habitaciones_y_deberian)

df.loc[(df.habitaciones.isnull())&(~df.tipodepropiedad.isin(sin_habitaciones_y_deberian)), 'habitaciones'] = 0

for p in sin_habitaciones_y_deberian:
    df.loc[(df.habitaciones.isnull()) & (df.tipodepropiedad == p), 'habitaciones'] = resumen_habitacion.loc[p, 'moda']


print("Quedaron {} con valor NA en banos".format(sum(df.habitaciones.isnull())))

Cantidad de propiedades sin dato del baño: 22453
['Apartamento', 'Casa', 'Casa en condominio', 'Casa uso de suelo', 'Departamento Compartido', 'Duplex', 'Otros', 'Quinta Vacacional', 'Rancho', 'Villa']
Quedaron 0 con valor NA en banos


### Antiguedad como NA

In [16]:
resumen_antiguedad

,con,sin,porcentaje_no,promedio_del_valor,moda
tipodepropiedad,,,,,
Apartamento,48290.0,9051,15.784517,8.362021,0.0
Bodega comercial,1129.0,277,19.701280,9.635961,10.0
Casa,119026.0,22691,16.011488,8.272588,0.0
Casa en condominio,16286.0,3011,15.603462,6.328626,0.0
Casa uso de suelo,616.0,92,12.994350,17.998377,20.0
Departamento Compartido,134.0,7,4.964539,6.149254,0.0
Duplex,337.0,6,1.749271,9.851632,10.0
Edificio,1201.0,195,13.968481,15.890924,20.0
Garage,NaN,1,100.000000,NaN,0.0


In [17]:
print("Cantidad de propiedades sin dato de antiguedad:", sum(df.antiguedad.isnull()))

reemplazar_por_moda(df, 'antiguedad', resumen_antiguedad)

print("Quedaron {} con valor NA en antiguedad".format(sum(df.antiguedad.isnull())))

Cantidad de propiedades sin dato de antiguedad: 43519
Quedaron 0 con valor NA en antiguedad


### Metro totales y metros cubiertos

Aca hay algunos casos medio "obvios".

Si metros totales esta vacio y tiene metros cubiertos, entonces los igualamos.

Si tiene metros totales pero no tiene metros cubiertos dependera del tipo de propiedad

In [18]:
print("Cantidad con NA en metroscubiertos y metrostotales:", sum((df.metroscubiertos.isnull()) & (df.metrostotales.isnull())))

Cantidad con NA en metroscubiertos y metrostotales: 0


Genial, no hay ningun caso con ambos vacios

In [19]:
print("Con metroscubiertos y sin metrostotales:", sum((~df.metroscubiertos.isnull()) & (df.metrostotales.isnull())))

Con metroscubiertos y sin metrostotales: 51458


In [20]:
df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metrostotales'] = df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metroscubiertos']

In [21]:
print("Con metroscubiertos y sin metrostotales:", sum((~df.metroscubiertos.isnull()) & (df.metrostotales.isnull())))

Con metroscubiertos y sin metrostotales: 0


Despues de igualar metrostotales a metroscubiertos ya no queda ningún dato en metrostotales sin valor.

In [22]:
print("Sin metroscubiertos y con metrostotales:", sum((df.metroscubiertos.isnull()) & (~df.metrostotales.isnull())))

Sin metroscubiertos y con metrostotales: 17386


In [23]:
df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metroscubiertos'] = df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metrostotales']

In [24]:
print("Sin metroscubiertos y con metrostotales:", sum((df.metroscubiertos.isnull()) & (~df.metrostotales.isnull())))

Sin metroscubiertos y con metrostotales: 0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239954 entries, 0 to 239999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tipodepropiedad  239954 non-null  object 
 1   provincia        239801 non-null  object 
 2   antiguedad       239954 non-null  float64
 3   habitaciones     239954 non-null  float64
 4   garages          239954 non-null  float64
 5   banos            239954 non-null  float64
 6   metroscubiertos  239954 non-null  float64
 7   metrostotales    239954 non-null  float64
 8   piscina          239954 non-null  float64
 9   precio           239954 non-null  float64
dtypes: float64(8), object(2)
memory usage: 20.1+ MB


# Toda la limpieza concentrada en un 

In [26]:
def limpieza_dataframe(df):
    # revisar si tiene sentido, sacamos 136 provincia y 500 sin tipodepropiedad
    df.dropna(subset=['tipodepropiedad', 'provincia'], inplace=True)
    
    # garages
    resumen_garages = pd.read_csv('r_garages.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'garages', resumen_garages)
    
    # baños
    resumen_banos = pd.read_csv('r_banos.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'banos', resumen_banos)
    
    # habitaciones
    resumen_habitacion = pd.read_csv('r_habitaciones.csv', index_col='tipodepropiedad')
    
    sin_habitaciones_y_deberian = list(resumen_habitacion[resumen_habitacion.porcentaje_no < 50].index)

    df.loc[(df.habitaciones.isnull())&(~df.tipodepropiedad.isin(sin_habitaciones_y_deberian)), 'habitaciones'] = 0

    for p in sin_habitaciones_y_deberian:
        df.loc[(df.habitaciones.isnull()) & (df.tipodepropiedad == p), 'habitaciones'] = resumen_habitacion.loc[p, 'moda']
    
    # antiguedad
    resumen_antiguedad = pd.read_csv('r_antiguedad.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'antiguedad', resumen_antiguedad)

    # metrostotales y metroscubiertos
    df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metrostotales'] = df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metroscubiertos']
    df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metroscubiertos'] = df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metrostotales']


In [27]:
nuevo_df = df_original.drop(drop_columns, axis=1)
nuevo_df.dropna(subset=['tipodepropiedad', 'provincia'], inplace=True)
nuevo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239801 entries, 0 to 239999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tipodepropiedad  239801 non-null  object 
 1   provincia        239801 non-null  object 
 2   antiguedad       196384 non-null  float64
 3   habitaciones     217368 non-null  float64
 4   garages          202138 non-null  float64
 5   banos            213684 non-null  float64
 6   metroscubiertos  222441 non-null  float64
 7   metrostotales    188389 non-null  float64
 8   piscina          239801 non-null  float64
 9   precio           239801 non-null  float64
dtypes: float64(8), object(2)
memory usage: 20.1+ MB


In [28]:
limpieza_dataframe(nuevo_df)
nuevo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239801 entries, 0 to 239999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tipodepropiedad  239801 non-null  object 
 1   provincia        239801 non-null  object 
 2   antiguedad       239801 non-null  float64
 3   habitaciones     239801 non-null  float64
 4   garages          239801 non-null  float64
 5   banos            239801 non-null  float64
 6   metroscubiertos  239801 non-null  float64
 7   metrostotales    239801 non-null  float64
 8   piscina          239801 non-null  float64
 9   precio           239801 non-null  float64
dtypes: float64(8), object(2)
memory usage: 20.1+ MB


In [29]:
pd.read_csv('r_antiguedad.csv', index_col='tipodepropiedad')
# generar_resumen(df, 'antiguedad')

,con,sin,porcentaje_no,promedio_del_valor,moda
tipodepropiedad,,,,,
Apartamento,48290.0,9051,15.784517,8.362021,0.0
Bodega comercial,1129.0,277,19.701280,9.635961,10.0
Casa,119026.0,22691,16.011488,8.272588,0.0
Casa en condominio,16286.0,3011,15.603462,6.328626,0.0
Casa uso de suelo,616.0,92,12.994350,17.998377,20.0
Departamento Compartido,134.0,7,4.964539,6.149254,0.0
Duplex,337.0,6,1.749271,9.851632,10.0
Edificio,1201.0,195,13.968481,15.890924,20.0
Garage,NaN,1,100.000000,NaN,0.0
